In [3]:
import pandas
import sklearn.metrics
import torch
import torch.nn as nn
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import importlib
import src.params as params
import src.model as model
import src.main as main
import src.dataloader as dataloader
importlib.reload(dataloader)
importlib.reload(model)
import torch.nn.functional as F
import src.dataloader as dataloader
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler

In [2]:
args = params.get_args("../config/system_a.yaml")
data,kpi_name = main.get_cmdb(args)
train,valid,test = main.get_dataset(args)
basic = model.Basic(args, data, kpi_name)

Loading preprocessed data from /home/junetheriver/code/aiops/workdir/system_a/data_pre.pkl
metric_num: 2152, kpi_num: 121, torch.Size([38880, 2153])


In [4]:
df = pd.read_csv("../../workdir/system_a/ts.csv")

In [5]:
df.head()

,Unnamed: 0,id,index,system.io.await,system.net.bytes_rcvd,system.net.udp.in_datagrams,system.net.bytes_sent,system.io.wrqm_s,system.net.tcp.out_segs,haproxy.backend.session.time,...,weblogic.webapp.sessions,system.disk.pct_usage,haproxy.backend.session.rate,haproxy.frontend.response.2xx,redis.mem.used,system.mem.real.used,system.fs.inodes.in_use,haproxy.frontend.requests.rate,system.run_time,system.load.5
0,0,0,0,0.0,2834.32,1.03,6207.08,0.0,47.13,0.0,...,0.0,47.20,0.0,0.0,0.0,10439.0,1.18,0.0,3731006.2,0.10
1,1,0,1,0.0,7579.49,1.05,4740.14,0.0,47.15,0.0,...,0.0,47.20,0.0,0.0,0.0,10181.0,1.18,0.0,3731065.5,0.11
2,2,0,2,0.0,4868.15,1.05,8800.13,0.0,54.05,0.0,...,0.0,47.20,0.0,0.0,0.0,10195.0,1.18,0.0,3731125.5,0.11
3,3,0,3,0.0,9075.92,1.07,15296.17,0.0,72.95,0.0,...,0.0,47.20,0.0,0.0,0.0,10071.0,1.18,0.0,3731185.8,0.09
4,4,0,4,0.0,7447.75,1.08,13647.97,0.0,68.58,0.0,...,0.0,47.21,0.0,0.0,0.0,10071.0,1.18,0.0,3731246.0,0.07


In [6]:
ss = StandardScaler()

In [7]:
values = df.values

In [13]:
values[:,3:] = ss.fit_transform(values[:,3:])

In [11]:
ss.mean_

array([4.31858612e-01, 1.52254536e+05, 9.20855270e+00, 2.12338312e+05,
       3.50841722e+01, 5.22402470e+02, 4.52544987e+00, 2.17646553e+02,
       4.40293573e+03, 4.16228792e-01, 4.92151535e+02, 6.23494936e+03,
       2.90898792e+01, 9.14601542e-02, 2.05398458e-01, 4.51573522e+00,
       8.72826403e+07, 1.82776350e-01, 5.64831877e+00, 4.47377892e-02,
       2.00514139e-02, 1.47117352e+01, 9.25449871e-01, 1.57986237e+03,
       3.85604113e-03, 4.35604113e-02, 4.20895617e+02, 1.90290488e-01,
       4.46940874e-02, 1.28534704e-03, 9.74989717e+01, 1.98200514e-01,
       7.22236504e-02, 5.54493573e-01, 4.29803342e+00, 4.46940874e-02,
       3.08483290e-03, 5.08309693e+07, 9.55309434e+01, 4.21198715e+00,
       4.29305913e-02, 5.26023907e+02, 8.23120217e+07, 4.24575835e+00,
       4.77585907e+02, 1.79948586e-03, 4.57069409e-02, 9.30591260e-01,
       4.26953470e+00, 5.14138817e-02, 1.73706530e+04, 2.88678663e-01,
       2.96401028e-01, 2.84364756e+03, 1.97787817e+04, 7.90359382e+04,
      

In [19]:
new_df = DataFrame(values[:,1:])

In [20]:
new_df.head()

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0.0,0.0,-0.125178,-0.567428,-0.261788,-0.608013,-0.208925,-0.602919,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.615230,-0.292515,-0.260023,0.557870,-0.280821
1,0.0,1.0,-0.125178,-0.549408,-0.261148,-0.612340,-0.208925,-0.602893,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.552438,-0.292515,-0.260023,0.557951,-0.265503
2,0.0,2.0,-0.125178,-0.559704,-0.261148,-0.600364,-0.208925,-0.594140,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.555846,-0.292515,-0.260023,0.558032,-0.265503
3,0.0,3.0,-0.125178,-0.543725,-0.260508,-0.581203,-0.208925,-0.570164,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.525667,-0.292515,-0.260023,0.558114,-0.296139
4,0.0,4.0,-0.125178,-0.549908,-0.260188,-0.586065,-0.208925,-0.575708,-0.100101,-0.109432,...,-0.782101,0.914229,-0.262319,-0.24826,-0.071888,0.525667,-0.292515,-0.260023,0.558196,-0.326775


In [23]:
new_df.columns = df.columns[1:]

In [24]:
new_df.head()

,id,index,system.io.await,system.net.bytes_rcvd,system.net.udp.in_datagrams,system.net.bytes_sent,system.io.wrqm_s,system.net.tcp.out_segs,haproxy.backend.session.time,system.io.wkb_s,...,weblogic.webapp.sessions,system.disk.pct_usage,haproxy.backend.session.rate,haproxy.frontend.response.2xx,redis.mem.used,system.mem.real.used,system.fs.inodes.in_use,haproxy.frontend.requests.rate,system.run_time,system.load.5
0,0.0,0.0,-0.125178,-0.567428,-0.261788,-0.608013,-0.208925,-0.602919,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.615230,-0.292515,-0.260023,0.557870,-0.280821
1,0.0,1.0,-0.125178,-0.549408,-0.261148,-0.612340,-0.208925,-0.602893,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.552438,-0.292515,-0.260023,0.557951,-0.265503
2,0.0,2.0,-0.125178,-0.559704,-0.261148,-0.600364,-0.208925,-0.594140,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.555846,-0.292515,-0.260023,0.558032,-0.265503
3,0.0,3.0,-0.125178,-0.543725,-0.260508,-0.581203,-0.208925,-0.570164,-0.100101,-0.109432,...,-0.782101,0.913373,-0.262319,-0.24826,-0.071888,0.525667,-0.292515,-0.260023,0.558114,-0.296139
4,0.0,4.0,-0.125178,-0.549908,-0.260188,-0.586065,-0.208925,-0.575708,-0.100101,-0.109432,...,-0.782101,0.914229,-0.262319,-0.24826,-0.071888,0.525667,-0.292515,-0.260023,0.558196,-0.326775


In [25]:
new_df.to_csv("../../workdir/system_a/ts_scaled.csv")

In [27]:
new_df.id.max()
ts = new_df

In [29]:
des_l = []
for idx in range(int(ts.id.max())+1):
    des_df = ts[ts.id == idx].describe()[1:].iloc[:,2:]
    des_l.append(des_df)

In [38]:
des_l[15]

,system.io.await,system.net.bytes_rcvd,system.net.udp.in_datagrams,system.net.bytes_sent,system.io.wrqm_s,system.net.tcp.out_segs,haproxy.backend.session.time,system.io.wkb_s,redis.replication.backlog_histlen,system.io.w_await,...,weblogic.webapp.sessions,system.disk.pct_usage,haproxy.backend.session.rate,haproxy.frontend.response.2xx,redis.mem.used,system.mem.real.used,system.fs.inodes.in_use,haproxy.frontend.requests.rate,system.run_time,system.load.5
mean,-0.007785,-0.509995,-0.261244,-0.540165,-0.138358,-0.512312,-0.100101,-0.059153,-7.188846e-02,-0.003192,...,2.967302,0.642663,-0.262319,-2.482596e-01,-0.071888,0.826166,-2.925148e-01,-0.260023,0.203253,-0.210359
std,0.142685,0.013983,0.000724,0.020584,0.148430,0.022595,0.000000,0.061875,1.462847e-17,0.139927,...,0.000071,0.000442,0.000000,2.925695e-17,0.000000,0.022790,5.851389e-17,0.000000,0.000247,0.034023
min,-0.125178,-0.534014,-0.262108,-0.577101,-0.208925,-0.549867,-0.100101,-0.109432,-7.188846e-02,-0.118316,...,2.967227,0.642150,-0.262319,-2.482596e-01,-0.071888,0.801171,-2.925148e-01,-0.260023,0.202886,-0.250185
25%,-0.125178,-0.521613,-0.261788,-0.554900,-0.208925,-0.529823,-0.100101,-0.109432,-7.188846e-02,-0.118316,...,2.967227,0.642150,-0.262319,-2.482596e-01,-0.071888,0.805978,-2.925148e-01,-0.260023,0.203070,-0.234867
50%,-0.059960,-0.505733,-0.261468,-0.535669,-0.208925,-0.507750,-0.100101,-0.092840,-7.188846e-02,-0.054358,...,2.967335,0.643005,-0.262319,-2.482596e-01,-0.071888,0.825874,-2.925148e-01,-0.260023,0.203253,-0.211891
75%,0.055259,-0.499491,-0.260668,-0.527118,-0.200737,-0.497376,-0.100101,-0.012644,-7.188846e-02,0.058634,...,2.967335,0.643005,-0.262319,-2.482596e-01,-0.071888,0.831533,-2.925148e-01,-0.260023,0.203437,-0.192743
max,0.228450,-0.493870,-0.260188,-0.512317,0.204945,-0.478091,-0.100101,0.043418,-7.188846e-02,0.228477,...,2.967436,0.643005,-0.262319,-2.482596e-01,-0.071888,0.863476,-2.925148e-01,-0.260023,0.203621,-0.158278


In [39]:
des_tensor = []
for df in des_l:
    des_tensor.append(torch.tensor(df.values).flatten())
statistical_features = torch.stack(des_tensor)

In [41]:
statistical_features.shape

torch.Size([389, 679])

In [42]:
torch.save(statistical_features, "../../workdir/system_a/stat_feature_scaled.txt")

In [43]:
import scipy

In [44]:
from scipy.fft import fft, ifft

In [59]:
a = abs(torch.fft.fft(torch.tensor(new_df.values[:10,2:]))).to(torch.float32)

In [51]:
new_df.values.shape

(3890, 99)

In [55]:
a.shape

torch.Size([10, 97])

In [60]:
l1 = nn.Linear(97, 10)
l1(a)

tensor([[ 0.7048, -2.3714,  4.5877,  2.6997, -2.0173,  2.9363, -1.2145, -0.3125,
         -0.1531,  0.0144],
        [ 0.6127, -2.3813,  4.6752,  2.6446, -1.9629,  2.8087, -1.1502, -0.3813,
         -0.1658, -0.0432],
        [ 0.6066, -2.4083,  4.6588,  2.7244, -1.9962,  2.7783, -1.1864, -0.3555,
         -0.1952,  0.0369],
        [ 0.6016, -2.5774,  4.5557,  2.8688, -2.0031,  2.5841, -1.1934, -0.3128,
         -0.2597,  0.0643],
        [ 0.6116, -2.5988,  4.5759,  2.9260, -2.0188,  2.5759, -1.1997, -0.3394,
         -0.2569,  0.0680],
        [ 0.6677, -2.4920,  4.5945,  2.8315, -1.9841,  2.7226, -1.2324, -0.3449,
         -0.1915,  0.1040],
        [ 0.6488, -2.5909,  4.5854,  2.8789, -2.0079,  2.6809, -1.2186, -0.3297,
         -0.2900,  0.0447],
        [ 0.6477, -2.6717,  4.6251,  2.9331, -2.0509,  2.6471, -1.2651, -0.2953,
         -0.2836,  0.0990],
        [ 0.5428, -2.6389,  4.7094,  2.8388, -1.9256,  2.5499, -1.2296, -0.3864,
         -0.3595,  0.0928],
        [ 0.7909, -